In [17]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from helpers import *
from implementations import reg_logistic_regression
from implementation_helpers import calculate_loss
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
DATA_TRAIN_PATH = 'train.csv'
y_or, tX_or, ids = load_csv_data(DATA_TRAIN_PATH)

In [4]:
y = y_or
tX = tX_or

In [55]:
tX = pd.DataFrame(tX)
tX.where(tX!=-999, inplace=True)
tX.fillna(tX.median(), inplace=True)    
tX = tX.values

In [5]:
y = y_for_logistic(y)

In [6]:
x_tr_tot, x_te_tot, y_tr_tot, y_te_tot = split_data(y,tX,0.8,1)

In [7]:
initial_w = np.zeros(x_tr_tot.shape[1])
max_iters = 10000
gamma = 1e-10
w, loss = logistic_regression(y_tr_tot, x_tr_tot, initial_w, max_iters, gamma)

NameError: name 'logistic_regression' is not defined

In [ ]:
y_pred = predict_labels(w, tx_te_tot, True)
acc = accuracy(y_te_tot, y_pred)
print(acc)

In [108]:
initial_w = np.zeros(x_tr_tot.shape[1])
lambda_ = 1e-8
max_iters = 10000
gamma = 1e-10
w, loss = reg_logistic_regression(y_tr_tot, x_tr_tot, lambda_, initial_w, max_iters, gamma)

Current iteration=0, loss=121300.75659799043
Current iteration=1000, loss=93581.46043394844
Current iteration=2000, loss=93239.80127423578
Current iteration=3000, loss=93127.35430130895
Current iteration=4000, loss=93077.86977980098
Current iteration=5000, loss=93052.77794496421
Current iteration=6000, loss=93038.0524237295
Current iteration=7000, loss=93027.87325478229
Current iteration=8000, loss=93019.71926021982
Current iteration=9000, loss=93012.4799203386


In [109]:
y_pred = predict_labels(w, x_te_tot, True)
acc = accuracy(y_te_tot, y_pred)
print(acc)

0.7349466666666666


In [15]:
def cross_validation(y, x, k_indices, k, lambda_, gamma, acc = False, ls = False):
    """return the loss of ridge regression."""
    losses_tr = []
    losses_te = []
    accuracies = []
    ws = []
    initial_w = np.zeros(x.shape[1])

    
    for k_group in range(k):
        index_te = k_indices[k_group]
        index_tr = np.setdiff1d(np.arange(len(y)), index_te)
        #print(index_te, index_tr)
        x_te = x[index_te]
        x_tr = x[index_tr]
        y_te = y[index_te]
        y_tr = y[index_tr]
        
        # form data with polynomial degree
        x_te_poly = x_te
        x_tr_poly = x_tr
        
        if (ls==False):
            w, _ = reg_logistic_regression(y_tr, x_tr_poly, lambda_, initial_w, 1000, gamma)
        else:
            w = least_squares_lstsq_ver(y_tr, x_tr_poly)
        
        # calculate the loss for train and test data
        rmse_tr = calculate_loss(y_tr, x_tr_poly, w)
        rmse_te = calculate_loss(y_te, x_te_poly, w)
        #print(lambda_, rmse_te)
        losses_tr.append(rmse_tr)
        losses_te.append(rmse_te)
        
        y_pred = predict_labels(w, x_te_poly)
        accuracies.append(accuracy(y_te, y_pred))
        
        ws.append(w)
        
    if acc==False:
        loss_tr = np.median(losses_tr)
        loss_te = np.median(losses_te)
        return loss_tr, loss_te, np.mean(ws, axis=0)
    else:
        return np.mean(accuracies), np.mean(ws, axis=0)

In [11]:
def select_best_hypers(y, x, k_fold, seed=1):
    #y_sub, x_sub = get_subsample(y, x, 100000, seed)
    lambdas = np.logspace(-8, 0, 8)
    gammas = np.logspace(-6, 0, 5)
    #lambdas=[1e-06]
    k_indices = build_k_indices(y, k_fold, seed)
    loss_min = np.inf
    gamma_star = 0
    lambda_star = 0
    w_star = 0
    for gamma in gammas:
        for lambda_ in lambdas:
            #print(degree)
            loss_tr, loss_te, w = cross_validation(y, x, k_indices, k_fold, lambda_, gamma)
            #print(loss_te)
            if loss_te < loss_min:
                loss_min = loss_te
                print("New loss: {}, gamma: {}, lambda: {}".format(loss_te, gamma, lambda_))
                gamma_star = gamma
                lambda_star = lambda_
                w_star = w
    return gamma_star, lambda_star, loss_min, w_star

In [ ]:
k_fold = 4
gammas_star=[]
lambdas_star=[]
w_star = []
for jet in range(0, 8):
    print("jet {}: ".format(jet))
    gamma_star, lambda_star, loss, w = select_best_hypers(y_tr[jet], tX_tr[jet], k_fold, 1)
    gammas_star.append(gamma_star)
    lambdas_star.append(lambda_star)
    w_star.append(w)
    #print("jet {}: Best accuracy {}, degree: {},  lambda: {}".format(jet, acc, degree_star, lambda_star))
    print("jet {}: best loss: {}, degree: {}, lambda: {}".format(jet, loss, gamma_star, lambda_star))

jet 0: 
New loss: 1223.9791281210773, gamma: 1e-06, lambda: 1e-08
New loss: 936.8265655823413, gamma: 3.1622776601683795e-05, lambda: 1e-08
jet 0: best loss: 936.8265655823413, degree: 3.1622776601683795e-05, lambda: 1e-08
jet 1: 
New loss: 8487.580246188481, gamma: 1e-06, lambda: 1e-08
jet 1: best loss: 8487.580246188481, degree: 1e-06, lambda: 1e-08
jet 2: 
New loss: 476.60760865688735, gamma: 1e-06, lambda: 1e-08
New loss: 418.492834146361, gamma: 3.1622776601683795e-05, lambda: 1e-08
jet 2: best loss: 418.492834146361, degree: 3.1622776601683795e-05, lambda: 1e-08
jet 3: 
New loss: 8371.862932929653, gamma: 1e-06, lambda: 1e-08


In [8]:
y_tr, tX_tr, indexes_tr, means_tr, std_tr = preprocessing(x_tr_tot, y_tr_tot)
y_te, tX_te, indexes_te, means_te, std_te = preprocessing(x_te_tot, y_te_tot)